In [1]:
import numpy as np
import matplotlib.pyplot as plt

In [24]:
from dataclasses import dataclass

# Dataclasses for SDK5 Configuration inputs parameteres.
@dataclass
class ChirpComnCfg:
    digOutputSampRate: float # Sampling Rate decimator (sampling rate given by 100MHz/digoutputsamprate i.e digoutsamplerate = 8, sampling rate = 12.5MHz)
    digOutputBitsSel: int # Digital output sample bits select:
    # n important pelo momento
    """ 
        - 0 - Digital sample output is 12 MSB bits of DFE after rounding 4 LSBs. 
        - 1 - Digital sample output is 12 bits after rounding 3 LSBs & clipping 1 MSB. 
        - 2 - Digital sample output is 12 bits after rounding 2 LSBs & clipping 2 MSB. 
        - 3 - Digital sample output is 12 bits after rounding 1 LSBs & clipping 3 MSB. 
        - 4 - Digital sample output is 12 LSB bits after clipping 4 MSB. 
        - 5 - Digital sample output is 16 bits.
    """
    dfeFirSel: int # The final stage FIR filter's characteristics:
    # n important pelo momento
    """
        0 - Long Filter (90% visibility)
        1 - Shorter filter (80% visibility)
    """
    numOfAdcSamples: int # Number of ADC samples collected during ADC sampling time (needs to be power of 2)
    
    chirpTxMimoPatSel: int # ? modelos de estimacao de angulo diferentes (n importante no momento)
    """
        1 - enable TDM
        4 - enable BPM
    """
    
    chirpRampEndTime: float # ramp end time (us) 
    # bem importante !!!!!!!!!!!!!!!!!!!!!!!!
    """
        Ramp end time (us): This parameter is the sum of the ADC start time, ADC sampling time, and some ramp excess time.
        In other words, its the ramp period.
        Documentation says a minimum of:
        (chirpAdcSkipSamples + numOfAdcSamples + 7), for short filter (dfeFirSel = 1)
        (chirpAdcSkipSamples + numOfAdcSamples + 10), for long filter (dfeFirSel = 0)

    """
    
    chirpRxHpfSel: int # Chirp Profile HPF corner frequency:
    # High pass filter to filter out the bumper or anything too close to the radar.
    """
        - 0 - 175kHz HPF corner frequency 
        - 1 - 350kHz HPF corner frequency
        - 2 - 700kHz HPF corner frequency 
        - 3 - 1400kHz HPF corner frequency
    """

@dataclass
class ChirpTimingCfg:
    chirpIdleTime: float # Idle Time (us) in the chirp cicle before the frequency ramp
    chirpAdcSkipSamples: int # Number of samples skipped during ADC sampling. The ADC start time in us is computed based on the sampling rate. (0 - 63)
    chirpTxStartTime: float # Tx Start Time (us). Recommended Range: +- 5us
    chirpRfFreqSlope: float # Frequency Slope (MHz/us). (-399MHz/us to +399MHz/us)
    chirpRfFreqStart: float # Starting Frequency (GHz)

@dataclass
class ChannelCfg:
    rxChannelEn: float # Rx Antenna Mask; for 3 antennas, it is 0111b = 7
    txChannelEn: float # Tx Antenna; for 2 antenas it is 0011b = 3
    miscCtrl: float # ? SoC cascading (not supported with SDK)

@dataclass
class FrameCfg:
    numOfChirppsInBurst: int # Number of chirps in a burst (1 - 65535)
    numOfChirpsAccum: int # Number of chirps accumulated (0 - 64)
    burstPeriodicity: float # Burst periodicity (us)
    numOfBurstsInFrame: int # Number of bursts within a frame (1 - 4096)
    framePeriodicity: float # Frame Periodicity (ms)
    numOfFrames: int # Number of total frames (0 - 65535) (0 = infinite)

@dataclass
class FactoryCalibCfg:
    saveEnable: bool # Save calibration data to flash (0-1)
    restoreEnable: bool # Restore and load calibration data from FLASH
    rxGain: float # RX Gain in dB (30-40 strongly recommended)
    txBackoff: float # TX Backoff in dB (0 - 26)
    flashOffset: float # Address ofset in the flash to be used while saving or restoring calibration data

# Dataclasses for device specific configuration parameters
# @dataclass
# class ConfigurationParameters:
#     StartFrequency: float # (GHz)
#     FrequencySlope: float # (MHz/us)
#     SamplingRate: float # (MHz)
#     NumberofSamples: int # Number of samples
#     NumberofChirpsLoops: int # Number of chirps loops
#     IdleTime: float # (us)
#     ADCValidStartTime: int # (us)
#     RampEndTime: float # (us)
#     FramePeriodicity: float # (ms)

class MmWaveConfig:
    def __init__(self, 
                 chirp_comn_cfg: ChirpComnCfg,
                 chirp_timing_cfg: ChirpTimingCfg,
                 channel_cfg: ChannelCfg,
                 frame_cfg: FrameCfg,
                 factory_calib_cfg: FactoryCalibCfg):
        
        self.chirp_comn_cfg = chirp_comn_cfg
        self.chirp_timing_cfg = chirp_timing_cfg
        self.channel_cfg = channel_cfg
        self.frame_cfg = frame_cfg
        self.factory_calib_cfg = factory_calib_cfg
        
        # Important parameters that come directly from the radar configuration (left here as comment as a path remember)
            # slope = mmwave_config.chirp_timing_cfg.chirpRfFreqSlope # MHz/us
            # f0 = mmwave_config.chirp_timing_cfg.chirpRfFreqStart # GHz
            # nADC = mmwave_config.chirp_comn_cfg.numOfAdcSamples # number of samples taken with the ADC (always power of two)
            # T_chirp_ramp = mmwave_config.chirp_comn_cfg.chirpRampEndTime # us # ramp duration
            # chirp_idle_time = mmwave_config.chirp_timing_cfg.chirpIdleTime # us
            
            # n_skiped_chirps = mmwave_config.chirp_timing_cfg.chirpAdcSkipSamples #skiped samples
            # n_chirps_in_burst = mmwave_config.frame_cfg.numOfChirppsInBurst
            # n_bursts_in_frame = mmwave_config.frame_cfg.numOfBurstsInFrame
            # T_frame = mmwave_config.frame_cfg.framePeriodicity # ms
            # T_burst = mmwave_config.frame_cfg.burstPeriodicity # us
            
        # Important parameters that dont come directly from the radar configuration
        self.SamplingFrequency = 100 / self.chirp_comn_cfg.digOutputSampRate # MHz # ADC freq
        self.IntermediateFrequencyMaximum = 0.9 * self.SamplingFrequency / 2 # MHz # Maximum readable frequency from the ADC in the current config
        self.TSampling = self.chirp_comn_cfg.numOfAdcSamples / self.SamplingFrequency # us # Period of sampling
        self.EndFreq = self.chirp_timing_cfg.chirpRfFreqStart + (self.chirp_timing_cfg.chirpRfFreqSlope * self.chirp_comn_cfg.chirpRampEndTime * 1e-3) # GHz
        
        self.Bandwidth = self.chirp_timing_cfg.chirpRfFreqSlope * self.TSampling # MHz
        
        self.TChirp = self.chirp_comn_cfg.chirpRampEndTime + self.chirp_timing_cfg.chirpIdleTime # us
        self.nChirpsLoop = self.frame_cfg.numOfChirppsInBurst * self.frame_cfg.burstPeriodicity # n chirps per frame
        
        self.ADCValidStartTime = self.chirp_timing_cfg.chirpAdcSkipSamples / self.SamplingFrequency # us
        
        self.CarrierFreq = self.chirp_timing_cfg.chirpRfFreqStart + ((self.chirp_timing_cfg.chirpRfFreqSlope * self.ADCValidStartTime) + (self.Bandwidth / 2)) * 1e-3 # GHz
        
        self.nRxAntenas = bin(int(channel_cfg.rxChannelEn)).count("1")
        self.nTxAntenas = bin(int(channel_cfg.txChannelEn)).count("1")
        self.nVirtualAntenas = self.nRxAntenas * self.nTxAntenas
        
        self.config_caracteristics = self.ConfigCaracteristics(self)
    
    @dataclass
    class ConfigCaracteristics:
        def __init__(self, mmwave_config: 'MmWaveConfig'):
            c = 3e2 # m/us
            
            self.RangeMaximum = mmwave_config.IntermediateFrequencyMaximum * c / (2 * mmwave_config.chirp_timing_cfg.chirpRfFreqSlope) # m
            self.RangeResolution = c / (2 * mmwave_config.Bandwidth) # m
            self.MaximumUnambiguousVelocity = c / (4 * mmwave_config.CarrierFreq * mmwave_config.TChirp * 1e-3) # m/s
            self.VelocityResolution = 2 * self.MaximumUnambiguousVelocity / (mmwave_config.frame_cfg.numOfBurstsInFrame) # m/s
            self.ChirpTime = mmwave_config.TSampling
            if mmwave_config.chirp_comn_cfg.chirpTxMimoPatSel == 4: # burst mode
                self.ChirpRepetitionPeriod = mmwave_config.frame_cfg.burstPeriodicity
            else: # mmwave_config.chirp_comn_cfg.chirpTxMimoPatSel == 1
                self.ChirpRepetitionPeriod = mmwave_config.nTxAntenas * mmwave_config.TChirp
            self.RFDutyCycle = mmwave_config.nChirpsLoop * mmwave_config.TChirp / mmwave_config.frame_cfg.framePeriodicity * 1e-2
            if mmwave_config.frame_cfg.numOfBurstsInFrame == 1:
                self.ComplianceChripTime = mmwave_config.TChirp * mmwave_config.nTxAntenas * mmwave_config.nChirpsLoop * 1e-2
            else:
                self.ComplianceChripTime = mmwave_config.frame_cfg.burstPeriodicity * mmwave_config.frame_cfg.numOfChirppsInBurst * 1e-2
            self.RadarCubeSize = None # to be implemented
            
        def __repr__(self):
            return (f"ConfigCaracteristics(\n"
                    f"  RangeMaximum: {self.RangeMaximum:.2f} m,\n"
                    f"  RangeResolution: {self.RangeResolution:.2f} m,\n"
                    f"  MaximumUnambiguousVelocity: {self.MaximumUnambiguousVelocity:.2f} m/s,\n"
                    f"  VelocityResolution: {self.VelocityResolution:.2f} m/s,\n"
                    f"  ChirpTime: {self.ChirpTime} us,\n"
                    f"  ChirpRepetitionPeriod: {self.ChirpRepetitionPeriod} us,\n"
                    f"  RFDutyCycle: {self.RFDutyCycle} %,\n"
                    f"  ComplianceChripTime: {self.ComplianceChripTime:.2f} ms,\n"
                    f"  RadarCubeSize: {self.RadarCubeSize} kB\n"
                    f")")

    def save_to_file(self, file_path: str = None):
        """Saves the configuration to a file in the required format."""
        if file_path is None:
            file_path = "./mmwave_config.cfg"
        
        with open(file_path, 'w') as file:
            file.write(f"% This cfg file needs to be completed with SDK commands\n")
            file.write(f"% The commands can be found at <SDK_BASE_DIR/docs/mmwave_sdk_user_guide.pdf\n")
            
            # Write the configurations in the required format
            file.write(f"chirpComnCfg {self.chirp_comn_cfg.digOutputSampRate} {self.chirp_comn_cfg.digOutputBitsSel} "
                    f"{self.chirp_comn_cfg.dfeFirSel} {self.chirp_comn_cfg.numOfAdcSamples} "
                    f"{self.chirp_comn_cfg.chirpTxMimoPatSel} {self.chirp_comn_cfg.chirpRampEndTime} "
                    f"{self.chirp_comn_cfg.chirpRxHpfSel}\n")
            
            file.write(f"chirpTimingCfg {self.chirp_timing_cfg.chirpIdleTime} {self.chirp_timing_cfg.chirpAdcSkipSamples} "
                    f"{self.chirp_timing_cfg.chirpTxStartTime} {self.chirp_timing_cfg.chirpRfFreqSlope} "
                    f"{self.chirp_timing_cfg.chirpRfFreqStart}\n")
            
            file.write(f"channelCfg {self.channel_cfg.rxChannelEn} {self.channel_cfg.txChannelEn} "
                    f"{self.channel_cfg.miscCtrl}\n")
            
            file.write(f"frameCfg {self.frame_cfg.numOfChirppsInBurst} {self.frame_cfg.numOfChirpsAccum} "
                    f"{self.frame_cfg.burstPeriodicity} {self.frame_cfg.numOfBurstsInFrame} "
                    f"{self.frame_cfg.framePeriodicity} {self.frame_cfg.numOfFrames}\n")
            
            file.write(f"factoryCalibCfg {self.factory_calib_cfg.saveEnable} {self.factory_calib_cfg.restoreEnable} "
                    f"{self.factory_calib_cfg.rxGain} {self.factory_calib_cfg.txBackoff} "
                    f"{self.factory_calib_cfg.flashOffset}\n")
    
    @classmethod
    def parse_mmwave_config(cls, file_content: str):
        """Parses the configuration from the provided file content."""
        lines = file_content.strip().split('\n')

        # Dictionaries to hold parsed values
        chirp_comn_cfg = chirp_timing_cfg = channel_cfg = frame_cfg = factory_calib_cfg = None

        # Parse each line and map values to the appropriate config object
        for line in lines:
            parts = line.split()
            if line.startswith("chirpComnCfg"):
                chirp_comn_cfg = ChirpComnCfg(*map(float, parts[1:]))
            elif line.startswith("chirpTimingCfg"):
                chirp_timing_cfg = ChirpTimingCfg(*map(float, parts[1:]))
            elif line.startswith("channelCfg"):
                channel_cfg = ChannelCfg(*map(float, parts[1:]))
            elif line.startswith("frameCfg"):
                frame_cfg = FrameCfg(*map(float, parts[1:]))
            elif line.startswith("factoryCalibCfg"):
                factory_calib_cfg = FactoryCalibCfg(*map(float, parts[1:]))

        # Create and return an instance of MmWaveConfig
        return cls(
            chirp_comn_cfg=chirp_comn_cfg,
            chirp_timing_cfg=chirp_timing_cfg,
            channel_cfg=channel_cfg,
            frame_cfg=frame_cfg,
            factory_calib_cfg=factory_calib_cfg
        )

def load_mmwave_config_from_file(file_path: str) -> MmWaveConfig:
    with open(file_path, 'r') as file:
        file_content = file.read()
    return MmWaveConfig.parse_mmwave_config(file_content)


In [25]:
# I'll first read the file content to understand its structure and then proceed to parse it.
file_path = './profile_2024_10_10T13_21_31_415.cfg'

# Parsing the configuration file
mmwave_config = load_mmwave_config_from_file(file_path)

mmwave_config.__dict__

{'chirp_comn_cfg': ChirpComnCfg(digOutputSampRate=33.0, digOutputBitsSel=0.0, dfeFirSel=0.0, numOfAdcSamples=128.0, chirpTxMimoPatSel=4.0, chirpRampEndTime=50.0, chirpRxHpfSel=0.0),
 'chirp_timing_cfg': ChirpTimingCfg(chirpIdleTime=6.0, chirpAdcSkipSamples=19.0, chirpTxStartTime=0.0, chirpRfFreqSlope=18.39, chirpRfFreqStart=76.0),
 'channel_cfg': ChannelCfg(rxChannelEn=7.0, txChannelEn=3.0, miscCtrl=0.0),
 'frame_cfg': FrameCfg(numOfChirppsInBurst=2.0, numOfChirpsAccum=0.0, burstPeriodicity=227.0, numOfBurstsInFrame=20.0, framePeriodicity=200.0, numOfFrames=0.0),
 'factory_calib_cfg': FactoryCalibCfg(saveEnable=1.0, restoreEnable=0.0, rxGain=30.0, txBackoff=0.0, flashOffset=0.0),
 'SamplingFrequency': 3.0303030303030303,
 'IntermediateFrequencyMaximum': 1.3636363636363638,
 'TSampling': 42.24,
 'EndFreq': 76.9195,
 'Bandwidth': 776.7936000000001,
 'TChirp': 56.0,
 'nChirpsLoop': 454.0,
 'ADCValidStartTime': 6.2700000000000005,
 'CarrierFreq': 76.5037021,
 'nRxAntenas': 3,
 'nTxAntenas'

In [4]:
# mmwave_config.save_to_file('./oi.cfg')
# mmwave_config.print_results(mmwave_config.config_caracteristics)

In [5]:
slope = mmwave_config.chirp_timing_cfg.chirpRfFreqSlope # MHz/us
f0 = mmwave_config.chirp_timing_cfg.chirpRfFreqStart # GHz
nADC = mmwave_config.chirp_comn_cfg.numOfAdcSamples # number of samples taken with the ADC (always power of two)
fs = 100/mmwave_config.chirp_comn_cfg.digOutputSampRate # MHz (ADC frequency)

IF_max = 0.9*fs/2 # MHz (max beat frequency as 90% of the Nyquist frequency)

T_sampling = nADC/fs # us
bandwidth = slope*T_sampling # MHz

T_chirp_ramp = mmwave_config.chirp_comn_cfg.chirpRampEndTime # ramp duration
chirp_idle_time = mmwave_config.chirp_timing_cfg.chirpIdleTime # us

f1 = f0 + (slope*T_chirp_ramp*1e-3) # GHz

T_chirp = chirp_idle_time + T_chirp_ramp # us

n_skiped_chirps = mmwave_config.chirp_timing_cfg.chirpAdcSkipSamples #skiped samples
ADC_valid_start_time = n_skiped_chirps/fs # us

fc = f0 + ((slope*ADC_valid_start_time) + (bandwidth/2))*1e-3 # GHz # its not the average of f0 and f1 because it should take in consideration that the ADC doesnt start with the chirps slope.

n_chirps_in_burst = mmwave_config.frame_cfg.numOfChirppsInBurst
n_bursts_in_frame = mmwave_config.frame_cfg.numOfBurstsInFrame
T_frame = mmwave_config.frame_cfg.framePeriodicity # ms
T_burst = mmwave_config.frame_cfg.burstPeriodicity # us

In [6]:
c = 3e2 # Speed of light (m/us) (to match the units of the slope)

Range_max = IF_max*c/(2*slope) # m (this is the max range limited by the bandwith of the IF, but it can also be limited by SNR)

print(f'Max Range: {Range_max} m')

T_sampling = nADC/fs # us
bandwidth = slope*T_sampling # MHz

Range_resolution = c/(2*bandwidth) # m

print(f'Range Resolution: {Range_resolution} m')

Max Range: 11.122645706658759 m
Range Resolution: 0.1931014879628256 m


In [7]:
# f0 = 310 # ??????????????????????????????????????????????????
carrier_wavelength = c/fc * 1e-3  # m
# carrier_wavelength = 3e8/(f0*1e9)

max_unambiguous_velocity = carrier_wavelength / (4*T_chirp*1e-6)

print(f'Max Unambiguous Velocity: {max_unambiguous_velocity} m/s')

# velocity_resolution = carrier_wavelength/(2*n_chir
# ps_in_burst*n_bursts_in_frame*T_chirp) # m/s

# print(f'Velocity Resolution: {velocity_resolution} m/s')

velocity_resolution = 2*max_unambiguous_velocity/(n_bursts_in_frame) # it should be 2 instead of 4 but its wrong (??)

print(f'Velocity Resolution: {velocity_resolution} m/s')

Max Unambiguous Velocity: 17.506155617607877 m/s
Velocity Resolution: 1.7506155617607877 m/s


In [8]:
print(f'Chirp Time, Tc (us): {T_sampling}')

if mmwave_config.chirp_comn_cfg.chirpTxMimoPatSel == 4: # burst mode
    print(f'Chirp Repetition Period, Tr (us): {T_burst}')
else: # chirpTxMimoPatSel == 1
    print(f'Chirp Repetition Period, Tr (us): {mmwave_config.nTxAntenas * T_chirp}')

Chirp Time, Tc (us): 42.24
Chirp Repetition Period, Tr (us): 227.0


In [9]:
n_chirps_loops = n_chirps_in_burst*n_bursts_in_frame # chirsp per frame
# if SDK5
print(f'RF Duty Cycle (%):{n_chirps_loops*T_chirp/T_frame*1e-1:.2f}')
if n_bursts_in_frame == 1:
    print(f'Compliance Chirp Time (ms): {T_chirp*mmwave_config.nTxAntenas*n_chirps_loops*1e-3}')
else:
    print(f'Compliance Chirp Time (ms): {T_burst*n_bursts_in_frame*1e-3}')
    
# if SDK3
# print(f'RF Duty Cycle (%):{mmwave_config.nTxAntenas*n_chirps_loops*T_chirp/T_frame*1e-1}')
# print(f'Compliance Chirp Time (ms): {T_chirp*mmwave_config.nTxAntenas*n_chirps_loops*1e-3}')

RF Duty Cycle (%):1.12
Compliance Chirp Time (ms): 4.54


In [10]:
# I belive this should be the same thing (??????)
print(f'Active Chriping Time (ms): {T_chirp_ramp*mmwave_config.nTxAntenas*n_chirps_loops*1e-3}') # SDK3
# print(f'Active Chriping Time (ms): {T_chirp_ramp*mmwave_config.nTxAntenas*n_chirps_in_burst*n_bursts_in_frame*1e-3}') #SDK5

# CUBE SIZE
# will make it later, just showing the ammount of memory it takes when all the calculations are done instead of calculating the teoric value (i couldnt make it work)


Active Chriping Time (ms): 4.0


In [11]:
print(f'Max Beat Frequency: {IF_max:.2f} MHz')
print(f'Valid Sweep Bandwidth, B (MHz): {bandwidth:.2f} MHz')

Max Beat Frequency: 1.36 MHz
Valid Sweep Bandwidth, B (MHz): 776.79 MHz


In [12]:
print(f'Carrier Frequency fc: {fc}')
print(f'End Frequency: {f1} GHz')

Carrier Frequency fc: 76.5037021
End Frequency: 76.9195 GHz
